In [ ]:
#FAQ
import grpc
from protos.nlp.faq import faq_pb2 as pb2
from protos.nlp.faq import faq_pb2_grpc as pb2_grpc
from grpc.beta import implementations
class Faq_inference():
    def __init__(self):
        super(Faq_inference, self).__init__()

    def faq_answer(self, query):
        # conn = grpc.insecure_channel('192.168.4.29:60009')
        conn = grpc.insecure_channel('172.31.208.10:58999')
        # conn = grpc.insecure_channel('172.31.208.10:58999')
        client = pb2_grpc.FaqStub(conn)
        res = client.GetFaq(pb2.FaqRequest(query=query, robot_id=1), timeout=1)
        answer = res.answer #faq对应的答案
        match = res.match   #匹配到的相似问句
        faq = res.faq       #对应的标准问句
        score = res.score   #匹配模型匹配的分数，可以视为置信度
        return answer, faq, score, match

if __name__ == "__main__":
    query = "我是你爸爸"
    tt = Faq_inference()
    print(tt.faq_answer(query))

In [ ]:
# 意图识别
import numpy as np
import pandas as pd
import joblib
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, confusion_matrix
import time

# 读取csv文件(前n行)
def read(train_set_path, test_set_path):
    data_1 = pd.read_csv('./faq.txt', sep='\t', header=None, names=['query', 'faq', 'answer','tts'], usecols = ['query'])
    data_1['class'] = 0
    data_temp = pd.read_csv('qingyun_chat.txt', header=None, names=['query'])
    data_temp['class'] = 1
    data_2 = pd.read_csv('./chat.txt', sep='\t', header=None, names=['match', 'query', 'answer','tts'], usecols = ['query'])
    data_2['class'] = 1
    data_3 = pd.read_csv('docqa.csv', header=None, names = ['query'])
    data_3['class'] = 2
    data_1 = data_1.drop_duplicates()
    data_2 = data_2.drop_duplicates()
    data = pd.concat((data_1, data_2, data_3, data_temp))
    
    data['query'] = data['query'].apply(lambda x: ' '.join(str(x)))
    X_train, X_test, y_train, y_test = train_test_split(data['query'], data['class'], test_size=0.15)
    return X_train, X_test, y_train, y_test

# 训练拟合模型
def train(X_train, y_train):
    vectorizer = TfidfVectorizer(max_features=800000, min_df=2, analyzer='word', token_pattern=u"(?u)\\b\\w+\\b", ngram_range=(1, 5)) 
    x_train = vectorizer.fit_transform(X_train)    
    # 构建逻辑回归模型
    logistic_model = LogisticRegression(C=3, dual=False) 
    # 拟合,训练
    logistic_model.fit(x_train, y_train) 
    
    tfidftransformer_path = 'models/tfidftransformer.pkl'
    with open(tfidftransformer_path, 'wb') as fw:
         pickle.dump(vectorizer, fw)        
    joblib.dump(logistic_model, "models/logisitic_model.pkl") 
    return vectorizer, logistic_model


# 保存预测结果
def save(test_data, y_test, save_path):
    # 保存结果
    y_test = y_test.tolist()  # 把(n,)的一维矩阵y_test转化为矩阵列表形式存储
    test_data['class'] = y_test + 1  # 新建一个class列,存入类别值(类别还原+1)
    data_result = test_data.loc[:, ['id', 'class']]  # 根据index索引所有行和id,class列
    data_result.to_csv(save_path, index=False)


if __name__=="__main__":
    chars = '*' * 20
    '''1.读取数据'''
    # old_time = time.time() 
    X_train, X_test, y_train, y_test = read('./faq.txt', './chat.txt')
    # current_time = time.time()
    # print(chars + '读取完成!用时{}s'.format(round(current_time-old_time,2)) + chars + '\n')

    '''2.训练'''
    vectorizer, logistic_model = train(X_train, y_train)
    
    vectorizer = pickle.load(open('models/tfidftransformer.pkl', "rb"))
    logistic_model = joblib.load('models/logisitic_model.pkl')
    
    '''3.预测'''
    x_test = vectorizer.transform(X_test)
    y_pred = logistic_model.predict(x_test) # (n,)
    acc_score = accuracy_score(y_test, y_pred)
    print(acc_score)
    F1_score = f1_score(y_test, y_pred, average='macro')
    print(F1_score)
    
    
    #0:闲聊 ， 1:客服， 2: 文档问答（知识问答）
    C2 = confusion_matrix(y_test, y_pred, labels=[0, 1, 2])
    # 计算百分比
    C2_normal = np.round(C2/np.sum(C2, axis=1).reshape(-1, 1), 2)

    sns.set()
    f, ax = plt.subplots()
    sns.heatmap(C2_normal, cmap="YlGnBu", annot=True, ax=ax) # heatmap

    #ax.set_title('Normalized confusion maxtrix') # title
    ax.set_xlabel('Predicted label')
    ax.set_ylabel('True label')
    ax.xaxis.set_ticklabels([0, 1, 2])
    ax.yaxis.set_ticklabels([0, 1, 2])
    
    print('Over!')

In [ ]:
#mapping = {-1:'Unknown', 0:'Faq', 1:'Chat', 2:'DocQA', 3:'Mutil-Turn'}
import pickle
import joblib
query = '她的妻子叫什么'
vectorizer = pickle.load(open('models/tfidftransformer.pkl', "rb"))
model = joblib.load('models/logisitic_model.pkl')
pred_class = model.predict(vectorizer.transform([' '.join(query)]))
pred_class

In [ ]:
import time
from protos.nlp.chat import chat_pb2
from protos.nlp.chat import chat_pb2_grpc
from grpc.beta import implementations


class Chat_inference(object):
    def __init__(self):
        super(Chat_inference, self).__init__()

    def response(self, history_utts, query):
        conn = implementations.insecure_channel("172.31.208.9", 60005)
        client = chat_pb2_grpc.ChatStub(channel=conn._channel)
        res = client.GetChat(chat_pb2.ChatRequest(history=history_utts, query=query, session_id='1', robot_id=1), timeout=60)
        answer = res.answer
        score = res.score
        query = res.query
        sessionid = res.session_id
        return answer, score, query, sessionid


if __name__ == "__main__":
    chat_infer = Chat_inference()
    
    history_utts = [{"conversation": [],'intent':[],'slots':[],'entities':[],'task_id':[]}]
    query = "你好"
    start_time = time.time()
    results = chat_infer.response(history_utts, query)
    end_time = time.time()
    print('*' * 20 + '回复完成!用时{:.2f}s'.format(end_time - start_time) + '*' * 20)
    print('results is:', *results)

In [ ]:
#DocQa
from protos.nlp.docqa import docqa_pb2
from protos.nlp.docqa import docqa_pb2_grpc
from grpc.beta import implementations
from google.protobuf.json_format import MessageToJson

class DocQAInference(object):
    def __init__(self, config):
        self.host = config.get('host', "172.31.208.4")
        self.port = config.get('port', 60005)
        # self.port = config.get('port', 6082)
        self.timeout = config.get('timeout', 60)

    def docqa(self, index_id, query):
        conn = implementations.insecure_channel(self.host, self.port)
        client = docqa_pb2_grpc.DocqaStub(channel=conn._channel)
        res = client.GetDocqa(docqa_pb2.DocqaRequest(index_id=index_id, query=query, session_id='1', robot_id=1), timeout=self.timeout)
        return res

#‘诗句：‘夜来风雨声’的下一句是什么’
if __name__ == "__main__":
    query_index_list = [
        ("中国有多少个省", "wiki"),
    ]
    query_list = [item[0] for item in query_index_list]
    index_list = [item[1] for item in query_index_list]
    # index_list = index_list*25
    # query_list = query_list*25
    
    inference = DocQAInference({})
    inference.docqa(index_list[0], query_list[0])

    import numpy as np
    import time
    query_lens = []
    latency_list = []
    for index, query in zip(index_list, query_list):
        print(f"index={index}")
        print(f"query={query}")
        start_time = time.time()    
        docqa_rst = inference.docqa(index, query)
        time_cost = time.time() - start_time
        print(f"result={MessageToJson(docqa_rst, ensure_ascii=False, indent=2)}")
        latency_list.append(time_cost*1000)
        query_len = len(query)
        query_lens.append(query_len)
        
                
    print(f"mean of query len is {np.mean(query_lens)}")
    print(f"mean of latency is {np.mean(latency_list)}ms")
    print(f"0.95 of latency is {np.quantile(latency_list, 0.95)}ms")
    print(f"0.99 of latency is {np.quantile(latency_list, 0.99)}ms")
    
#特朗普的弟弟叫什么
#碧昂斯的出生地

In [ ]:
import json

with open("squad_data.json", encoding='utf-8', errors='ignore') as json_data:
     data = json.load(json_data, strict=False)

In [ ]:
data

In [ ]:
query = []
for i in range(len(data['data'])):
    for j in range(len(data['data'][i]['paragraphs'])):
        for z in data['data'][i]['paragraphs'][j]['qas']:
            query.append(z['question'])
            break

In [ ]:
import pandas as pd
df = pd.DataFrame(query)

In [ ]:
df.to_csv('docqa_1.csv', index=False, header=False)

In [ ]:
#改写模块
import grpc
import sys
from protos.nlp.rewrite import rewrite_pb2
from protos.nlp.rewrite import rewrite_pb2_grpc

import requests
import base64

from grpc.beta import implementations

class IUR_inference(object):
    def __init__(self, config):
        self.host = config.get('host', "192.168.68.26")
        self.port = config.get('port', 60005)
        self.timeout = config.get('timeout', 60)

    def iur(self, history_utts, query):
        conn = implementations.insecure_channel(self.host, self.port)
        client = rewrite_pb2_grpc.ReWriteStub(channel=conn._channel)
        res = client.GetReWrite(rewrite_pb2.ReWriteRequest(history=history_utts, query=query, session_id='1', robot_id=1), timeout=self.timeout)
        new_query = res.new_query
        score = res.score
        return new_query, score

    def query_write_strategy(self, query, conversation):
        new_query_1, qr_score_1 = self.iur(conversation[:2], query)
        new_query_2, qr_score_2 = self.iur(conversation[2:], query)
        new_query_3, qr_score_3 = self.iur(conversation, query)
        new_query, qr_score = new_query_1, qr_score_1
        if qr_score_1 <qr_score_2:
            new_query = new_query_2
            qr_score = qr_score_2
        print(new_query_1, qr_score_1)
        print(new_query_2, qr_score_2)
        print(new_query_3, qr_score_3)
        if qr_score>0.8 and new_query!=query:
            return new_query
        return query
    
if __name__ == "__main__":
    # history_utts = ["", "", "北京可以办理工作居住证么？", "可以，根据政府政策规定，入职满一年后满足条件的可办理。"]
    # query = "多久能下来？"
    # history_utts = ["", "", "有松下的吹风机吗", "没有哦亲"]
    # query = "那有戴森的吗"
    # history_utts = ["", "", "帮我放周杰伦的歌", "好的，正在播放周杰伦的稻香"]
    # query = "算了，不要放他的，放张学友的"
    # history_utts = ['','']
    # history_utts = ['你知道碧昂斯吗', '肯定知道啊','那你知道她的第一张专辑吗','危险爱情']
    # history_utts = ['你知道碧昂斯吗','我的朋友有很多啊，你也是我朋友啊你忘啦？', '她的第一张专辑是什么', '危险爱情']
    history_utts = ['','','今天的天气怎么样', '阴天,有点闷热']
    # history_utts = ['怎么申请设备呢', 'IT标准库存资产在金蝶系统-人人资产填写申请；IT标准配件在飞书-工作台-审批填写耗材领用流程；非标准库存配件在金蝶系统填写采购申请。', '那北京可以办理居住证吗', '可以，根据政府政策规定，入职满一年后满足条件的可办理。']
    # history_utts = ['怎么申请设备', 'IT标准库存资产在金蝶系统-人人资产填写申请；IT标准配件在飞书-工作台-审批填写耗材领用流程；非标准库存配件在金蝶系统填写采购申请。', '北京可以办理居住证吗', '可以，需上家公司减员后绑定公司关联码后系统提交材料即可。']
    # history_utts = ['你好', '你好呀，我可以帮你做点啥嘛', '你认识碧昂斯吗', '那就多了去了，亲人朋友名人数不胜数呐！']
    # query="她的第一张专辑是什么"
    query = '明天的呢'
    tt = IUR_inference({})
    new_query = tt.query_write_strategy(query, history_utts)
    # new_query, score = tt.iur(history_utts, query)
    print(new_query)
#0.9014959335327148

In [ ]:
import re
from datetime import datetime, timedelta
from dateutil.parser import parse
import jieba.posseg as psg
import jieba
jieba.add_word('今天', 100000)
jieba.add_word('明天', 100000)
jieba.add_word('后天', 100000)
jieba.add_word('上午',100000)
jieba.add_word('下午',100000)

def get_time_word(text):
    time_res = []
    word = ''
    keyDate = {'今天': 0, '明天':1, '后天': 2}
    for k, v in psg.cut(text):
        if k in keyDate:
            if word != '':
                time_res.append(word)
            time_res.append(k)
        elif word != '':
            if v in ['m', 't']:
                word = word + k
            else:
                time_res.append(word)
                word = ''
        elif v in ['m', 't'] and '张' not in k:
            word = k
    if word != '':
        time_res.append(word)
    return time_res

def time_extract(text):
    # text = text
    time_res = []
    word = ''
    keyDate = {'今天': 0, '明天':1, '后天': 2}
    for k, v in psg.cut(text):
        if k in keyDate:
            if word != '':
                time_res.append(word)
            word = (datetime.today() + timedelta(days=keyDate.get(k, 0))).strftime('%Y年%m月%d日')
        elif word != '':
            if v in ['m', 't']:
                word = word + k
            else:
                time_res.append(word)
                word = ''
        elif v in ['m', 't'] and '张' not in k:
            word = k
    if word != '':
        time_res.append(word)
    result = list(filter(lambda x: x is not None, [check_time_valid(w) for w in time_res]))
    final_res = [parse_datetime(w) for w in result]
    return [x for x in final_res if x is not None]

def check_time_valid(word):
    m = re.match("\d+$", word)
    if m:
        if len(word) <= 6:
            return None
    word1 = re.sub('[号|日]\d+$', '日', word)
    if word1 != word:
        return check_time_valid(word1)
    else:
        return word1
    
def parse_datetime(msg):
    if msg is None or len(msg) == 0:
        return None

    if '午' not in msg and '点' not in msg:
        dt = parse(msg, fuzzy=True)
        return dt.strftime('%Y-%m-%d %H:%M:%S')
    else:
        m = re.match(
            r"([0-9零一二两三四五六七八九十]+年)?([0-9一二两三四五六七八九十]+月)?([0-9一二两三四五六七八九十]+[号日])?([上中下午晚早]+)?([0-9零一二两三四五六七八九十百]+[点:\.时])?([0-9零一二三四五六七八九十百]+分?)?([0-9零一二三四五六七八九十百]+秒)?",
            msg)
        if m.group(0) is not None:
            res = {
                "year": m.group(1),
                "month": m.group(2),
                "day": m.group(3),
                "hour": m.group(5) if m.group(5) is not None else '00',
                "minute": m.group(6) if m.group(6) is not None else '00',
                "second": m.group(7) if m.group(7) is not None else '00',
            }
            params = {}

            for name in res:
                if res[name] is not None and len(res[name]) != 0:
                    tmp = None
                    if name == 'year':
                        tmp = year2dig(res[name][:-1])
                    else:
                        tmp = cn2dig(res[name][:-1])
                    if tmp is not None:
                        params[name] = int(tmp)
            target_date = datetime.today().replace(**params)
            is_pm = m.group(4)
            if is_pm is not None:
                if is_pm == u'下午' or is_pm == u'晚上' or is_pm =='中午':
                    hour = target_date.time().hour
                    if hour < 12:
                        target_date = target_date.replace(hour=hour + 12)
            return target_date.strftime('%Y-%m-%d %H:%M:%S')
        else:
            return None
        
UTIL_CN_NUM = {
    '零': 0, '一': 1, '二': 2, '两': 2, '三': 3, '四': 4,
    '五': 5, '六': 6, '七': 7, '八': 8, '九': 9,
    '0': 0, '1': 1, '2': 2, '3': 3, '4': 4,
    '5': 5, '6': 6, '7': 7, '8': 8, '9': 9
}
UTIL_CN_UNIT = {'十': 10, '百': 100, '千': 1000, '万': 10000}
def cn2dig(src):
    if src == "":
        return None
    m = re.match("\d+", src)
    if m:
        return int(m.group(0))
    rsl = 0
    unit = 1
    for item in src[::-1]:
        if item in UTIL_CN_UNIT.keys():
            unit = UTIL_CN_UNIT[item]
        elif item in UTIL_CN_NUM.keys():
            num = UTIL_CN_NUM[item]
            rsl += num * unit
        else:
            return None
    if rsl < unit:
        rsl += unit
    return rsl
def year2dig(year):
    res = ''
    for item in year:
        if item in UTIL_CN_NUM.keys():
            res = res + str(UTIL_CN_NUM[item])
        else:
            res = res + item
    m = re.match("\d+", res)
    if m:
        if len(m.group(0)) == 2:
            return int(datetime.datetime.today().year/100)*100 + int(m.group(0))
        else:
            return int(m.group(0))
    else:
        return None

In [ ]:
import re
def ner(query):
    entities = []
    slots = []
    query = query.replace('今晚','今天晚上').replace('明早','明天早上').replace('明晚','明天晚上')

    time = time_extract(query)
    if time:
        slots.append('time')
        entities.append(time[0])
    time_word = get_time_word(query)
    for word in time_word:
        query = query.replace(word, '')
    print(query)

    m = re.findall(r"订.*?[0-9一二两三四五六七八九十]?张?从?([\u4e00-\u9fa5]{2,5}?)?(出发)?[到｜飞|去｜]([\u4e00-\u9fa5]{2,5}?)的",query)
    if m:
        start_city = m[0][0]
        end_city = m[0][2]
        if start_city:
            slots.append('start_city')
            entities.append(start_city)
        if end_city:
            slots.append('end_city')
            entities.append(end_city)

        print(start_city)
        print(end_city)
    else:
        pass
    return slots, entities
    
ner('明晚八点')
# ner('订一张明晚八点的机票')

In [ ]:
text1 = '帮我订一张明晚八点上海到深圳的机票'
print(text1, time_extract(text1), sep=':')

# text2 = '预定28号的房间'
# print(text2, time_extract(text2), sep=':')

# text3 = '我要从26号下午4点住到11月2号'
# print(text3, time_extract(text3), sep=':')

In [ ]:
#情绪识别

import grpc
from protos.nlp.faq import faq_pb2, faq_pb2_grpc
from protos.nlp.emotion import emotion_pb2, emotion_pb2_grpc

channel = grpc.insecure_channel('172.31.208.10:58999')
texts = ['你真棒']

# emotion
emotion_client = emotion_pb2_grpc.EmotionStub(channel)
for query in texts:
    resp = emotion_client.GetEmotion(emotion_pb2.EmotionRequest(query=query, session_id='1', robot_id=2))
    # print(resp.answer.happy, resp.answer.sad, resp.answer.angry, resp.answer.disgust,
    #       resp.answer.fear, resp.answer.surprise, resp.answer.none)
    print(resp)
    print()

In [2]:
#领域分类模型（Fasttext）
import fasttext
import jieba
# from gensim.models import FastText
jieba.add_word('你知道',100000)

model = fasttext.train_supervised(input = r"./train.txt", label_prefix="__label__", epoch=10, verbose=2, lr=0.04, loss="softmax")

model.save_model(r"models/model.bin")

stop_words = ['的','了','吗','啊','拉','呢','你知道','告诉', '请问']

def word_segment(text):
    seg_list = jieba.cut(text)
    word_list = []
    for word in seg_list:
        if word not in stop_words:
            word_list.append(word)
    line = " ".join(word_list)
    print(line)
    return line

# 预测
model = fasttext.load_model(r"models/model.bin")
print(model.test(r"./test.txt")  )

# text = 'hi穆莎'
text = '世界最高的山峰是哪座'
print(model.predict(word_segment(text))[0][0][-1])

Read 0M words
Number of words:  57491
Number of labels: 3
Progress: 100.0% words/sec/thread: 2813381 lr:  0.000000 avg.loss:  0.068367 ETA:   0h 0m 0s


(17536, 0.9757641423357665, 0.9757641423357665)
世界 最高 山峰 是 哪座
2


In [1]:
#文件预处理
import jieba
jieba.add_word('你知道',100000)
import pandas as pd
from sklearn.model_selection import train_test_split
#Faq
data_1 = pd.read_csv('./faq.txt', sep='\t', header=None, names=['query'])
data_1['class'] = 0

#闲聊
data_temp = pd.read_csv('qingyun_chat.txt', header=None, names=['query'])[:50000]
data_temp['class'] = 1
data_2 = pd.read_csv('./chat.txt', sep='\t', header=None, names=['query'])
data_2['class'] = 1

#文档问答
data_3 = pd.read_csv('docqa_2.csv', header=None, names = ['query'])
data_3['class'] = 2

data_2 = data_2.drop_duplicates()
data_temp = data_temp.drop_duplicates()
data = pd.concat((data_1, data_2, data_3, data_temp))
X_train, X_test, y_train, y_test = train_test_split(data['query'], data['class'], test_size=0.15)

stop_words = ['你知道','的','了','吗','啊','拉','呢','请问','啥']

def word_segment(text):
    seg_list = jieba.cut(text)
    word_list = []
    for word in seg_list:
        if word not in stop_words:
            word_list.append(word)
    line = " ".join(word_list)
    return line


#2: 文档问答。1:闲聊
with open('train.txt', 'w', encoding='utf-8') as f:
    for i in range(len(X_train)):
        f.write('__label__'+str(y_train.values[i])+'\t'+word_segment(X_train.values[i])+'\n')
        
f.close()

with open('test.txt', 'w', encoding='utf-8') as f:
    for i in range(len(X_test)):
        f.write('__label__'+str(y_test.values[i])+'\t'+word_segment(X_test.values[i])+'\n')
        
f.close()

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/0p/cjvwlkfs59d5t070gww_cp300000gn/T/jieba.cache
Loading model cost 0.281 seconds.
Prefix dict has been built successfully.


In [ ]:
import grpc
from protos.nlp.dm import dm_pb2
from protos.nlp.dm import dm_pb2_grpc

def run():
    # 连接 rpc 服务器
    channel = grpc.insecure_channel('localhost:50051')
    # 调用 rpc 服务
    stub = dm_pb2_grpc.DmStub(channel)
    history = {}
    while True:
        text = input()
        response = stub.GetDm(dm_pb2.DmRequest(text=text, history=history,session_id='1', robot_id = -1))
        print("Dm client received: " + response.answer)
        history = response.history

if __name__ == '__main__':
    run()

In [ ]:
import sys
import argparse 
sys.path.append('./DM/')
sys.path.append('./DM/models/')
from DmClass import *

text = '帮订一张从深圳到北京的机票'
history = {'conversation':['','']}

parser = argparse.ArgumentParser()
parser.add_argument('--robot_type', default='', choices=['DH', 'KF'], type=str, help='Robot type, DH or KF')
parser.add_argument('--chat_repo', default=1, choices=[0, 1], type=int, help='which repo used by chat_infer module')
parser.add_argaument('--skill_type', default='DH', choices=['DH', 'KF'], help='What type of multi-turn capability the robot has')
parser.add_argument('--FAQ', action="store_true", default=False, help='if use the faq server')
parser.add_argument('--QR', action="store_true", default=False, help='if use the query rewrite server')

parser.add_argument('--qr_ip', default='192.168.68.26', type=str)
parser.add_argument('--qr_port', default='60005', type=str)
parser.add_argument('--faq_ip', default='172.31.208.10', type=str)
parser.add_argument('--faq_port', default='58999', type=str)
parser.add_argument('--docqa_ip', default='172.31.208.4', type=str)
parser.add_argument('--docqa_port', default='60005', type=str)
parser.add_argument('--chat_ip', default='172.31.208.9', type=str)
parser.add_argument('--chat_port', default='60005', type=str)
parser.add_argument('--emotion_ip', default='172.31.208.10', type=str)
parser.add_argument('--emotion_port', default='58999', type=str)
args = parser.parse_args(args=[])
dm  = DmClass(args)
dm.load_data(text, history, '1', -1)

ans, history, emotion = dm.response()
print(ans)

In [3]:
#DM1.1
#闲聊、Faq、文档问答（wiki、mita）
import grpc
from protos.nlp.dm import dm_pb2
from protos.nlp.dm import dm_pb2_grpc

def run():
    # 连接 rpc 服务器
    channel = grpc.insecure_channel('172.31.208.4:55555')
    # 调用 rpc 服务
    stub = dm_pb2_grpc.DmStub(channel)
    
    #第一轮
    history = {}
    while True:
        text = input()
        print('text:',text)
        response = stub.GetDm(dm_pb2.DmRequest(text=text, history=history, session_id='1', robot_id = -1))
        print("Dm client received: " + response.answer,'\n')
        history = response.history
    
if __name__ == '__main__':
    run()

 今天天气怎么样 


text: 今天天气怎么样 


_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "failed to connect to all addresses"
	debug_error_string = "{"created":"@1679289779.143625000","description":"Failed to pick subchannel","file":"src/core/ext/filters/client_channel/client_channel.cc","file_line":3094,"referenced_errors":[{"created":"@1679289779.143624000","description":"failed to connect to all addresses","file":"src/core/lib/transport/error_utils.cc","file_line":163,"grpc_status":14}]}"
>

In [ ]:
import re
import requests, random, json
def tupe_conversation(conversation):
    result = []
    for i in range(0, len(conversation), 2):
        result.append((conversation[i], conversation[i+1]))
    return result

query = "你知道碧昂斯吗，回复中不要有英文字母"
conversation = ['你好', '你好👋!我是人工智能助手 ChatGLM-6B,很高兴见到你,欢迎问我任何问题。']

len_now = len(query)+sum(len(text) for text in conversation)
chat_params = {"max_length":len_now+50, "num_beams":1, "do_sample":True, "top_p":0.7, "temperature":0.95,
                "history":tupe_conversation(conversation)}
chat_params = json.dumps(chat_params, ensure_ascii=False)

session_hash = "".join(random.sample('zyxwvutsrqponmlkjihgfedcba1234567890',9))
resp = requests.post(url="http://172.31.208.4:60015/run/predict/", json={"data": [query, None, chat_params], "fn_index": 0, "session_hash": session_hash}).json()
print(re.match('<p>(.*?)</p>\n', resp['data'][4]['value'])[1])

ChatGLM-6B：碧昂丝(B胸痛)是一位知名的美国女歌手、词曲创作人和演员。她出生于1982年6月13日,出生地为美国加利福尼亚州洛杉矶市。碧昂丝以独特的声音、出色的舞蹈技巧和多样化的音乐风格而闻名


In [60]:
import re
re.match('<p>(.*?)</p>\n', resp['data'][4]['value'])[1]

'ChatGLM-6B：碧昂丝(Bailey Cyrus)是一位著名的美国女歌手、演员和词曲创作人。她出生于1982'